
#Amazon Review- Tagging Negative Review in Amazon Product Review with DistilBERT Model



#Introduction

### Connecting to Golab Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.1 MB/s eta 0:00:00


### Importing Libraries

In [3]:
import csv
import numpy as np
import pandas as pd
import torch
from sklearn import metrics
from torch import cuda
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import DistilBertModel, DistilBertTokenizer
import time

### Check Device

In [4]:
device = 'cuda' if cuda.is_available() else 'cpu'

### Dataset

In [10]:
df_train = "/content/drive/MyDrive/Masters_Thesis/Dataset/encoded_data/encoded_topic(seller)2subTopic_train.csv"
df_val = "/content/drive/MyDrive/Masters_Thesis/Dataset/encoded_data/encoded_topic(seller)2subTopic_test.csv"
target_list = "encode_sub_seller_topic"

In [11]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
train_loader, valid_loader = get_data_loaders(df_train, df_val, tokenizer, train_batch_size = 8, valid_batch_size = 8)

DistilBertTokenizer(name_or_path='distilbert-base-cased', vocab_size=28996, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)


### Model Implementation

In [12]:
model = DistilBERTClass()
model.to(device)
loss_function = torch.nn.CrossEntropyLoss()
LEARNING_RATE = 2e-5
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
train(total_epoch=20, model=model, train_loader=train_loader)

 Epoch  |  Train Loss  | Train Accuracy |  Elapsed 
---------------------------------------------------
   1    |   0.226136   |  91.325447   |   2.58   
   2    |   0.071209   |  98.149833   |   2.58   
   3    |   0.044126   |  98.574462   |   2.58   
   4    |   0.020475   |  99.484380   |   2.58   
   5    |   0.017307   |  99.423719   |   2.58   
   6    |   0.002887   |  99.909008   |   2.58   
   7    |   0.005849   |  99.909008   |   2.58   
   8    |   0.005430   |  99.878678   |   2.58   
   9    |   0.003799   |  99.848347   |   2.57   
  10    |   0.014249   |  99.514710   |   2.58   
  11    |   0.004839   |  99.757355   |   2.57   
  12    |   0.008348   |  99.787686   |   2.57   
  13    |   0.006833   |  99.848347   |   2.57   
  14    |   0.002210   |  99.909008   |   2.58   
  15    |   0.001536   |  99.878678   |   2.58   
  16    |   0.008287   |  99.818016   |   2.58   
  17    |   0.002136   |  99.939339   |   2.58   
  18    |   0.001384   |  99.939339   |   2.58

In [14]:
predictions = valid(model, valid_loader) 

 Valid Loss  | Valid Accuracy |  Elapsed 
-----------------------------------------
  0.087326   |   98.665049    |   0.23   


In [15]:
path = "/content/drive/MyDrive/Masters_Thesis/Models/DISTILBERT"
saveModel(model=model, path=path)

All files saved


In [16]:
saveCSVValidationResult(predictions,path)

### Data Preprocessing & DataLoader

In [5]:
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.reviewText
        self.tokenizer = tokenizer
        self.targets = df[target_list].values

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        #the attention_masks and token type ids, everything is returned in a dictionary format
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.long),
            'text' : text
        }

def get_data_loaders(train_dataframe, valid_dataframe, tokenizer, max_len=512, train_batch_size=16, valid_batch_size=16, learning_rate=2e-5):

    Train_Dataframe = pd.read_csv(train_dataframe)
    Validation_Dataframe = pd.read_csv(valid_dataframe)
    print(tokenizer)
    train_dataset = CustomDataset(Train_Dataframe, tokenizer, max_len)
    valid_dataset = CustomDataset(Validation_Dataframe, tokenizer, max_len)
    train_loader = DataLoader(train_dataset, batch_size=train_batch_size,
                              num_workers=1, shuffle=True, pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=valid_batch_size,
                              num_workers=1, shuffle=False, pin_memory=True)

    return train_loader, valid_loader

### DistilBERT Model

In [6]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

### Training & Validation


In [7]:

def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

def train(total_epoch, model, train_loader):
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Train Accuracy':^12} | {'Elapsed':^9}")
    print("-"*51)
    for epoch in range(total_epoch): 
      t0_epoch = time.time() 
      tr_loss = 0
      n_correct = 0
      nb_tr_steps = 0
      nb_tr_examples = 0
      model.train()
      for _,data in enumerate(train_loader, 0):
          ids = data['ids'].to(device, dtype = torch.long)
          mask = data['mask'].to(device, dtype = torch.long)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
          targets = data['targets'].to(device, dtype = torch.long)

          #TODO: make change for distilbert and bert - token_type_ids
          outputs = model(ids, mask, token_type_ids)
        
          loss = loss_function(outputs, targets)
          tr_loss += loss.item()
          big_val, big_idx = torch.max(outputs.data, dim=1)
          n_correct += calcuate_accu(big_idx, targets)

          nb_tr_steps += 1
          nb_tr_examples+=targets.size(0)
          
          optimizer.zero_grad()
          loss.backward()
          # # When using GPU
          optimizer.step()

      time_elapsed = (time.time() - t0_epoch)/60
      epoch_loss = tr_loss/nb_tr_steps
      epoch_accu = (n_correct*100)/nb_tr_examples
      print(f"{epoch + 1:^7} | {epoch_loss:^12.6f} | {epoch_accu:^12.6f} | {time_elapsed:^9.2f}")
 

def valid(model, testing_loader):
    print(f"{'Valid Loss':^12} | {'Valid Accuracy':^12} | {'Elapsed':^9}")
    print("-"*41)
    model.eval()
    t0_epoch = time.time() 
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
          ids = data['ids'].to(device, dtype = torch.long)
          mask = data['mask'].to(device, dtype = torch.long)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
          targets = data['targets'].to(device, dtype = torch.long)
          text = data['text']
          outputs = model(ids, mask, token_type_ids)


          loss = loss_function(outputs, targets)
          tr_loss += loss.item()
          big_val, big_idx = torch.max(outputs.data, dim=1)
          n_correct += calcuate_accu(big_idx, targets)

          nb_tr_steps += 1
          nb_tr_examples+=targets.size(0)

          for i in range(len(text)):
            predictions.append({
                'text': text[i],
                'predicted': switch_issue(big_idx[i].item()),
                'target': switch_issue(targets[i].item())
                })

    time_elapsed = (time.time() - t0_epoch)/60                
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"{epoch_loss:^12.6f} | {epoch_accu:^14.6f} | {time_elapsed:^9.2f}")
    return predictions

def switch_issue(issue_type):
    switcher = {
      1: 'Delivery and Return Issue',
      0: 'Product Authenticity Issue'
      }
    return switcher.get(issue_type, "Invalid Issue Type")

### Save Files

In [8]:
def saveModel(model, path):
  MODEL_PATH = path+'/Topic(Seller)2SubTopic/back_only_01/model.pth'
#   VOCAB_PATH = path+'/vocab'
  torch.save(model.state_dict(), MODEL_PATH)  
  print('All files saved')

def loadModel(path):
  MODEL_PATH = path+'/Topic(Seller)2SubTopic/back_only_01/model.pth'
#   VOCAB_PATH = path+'/vocab'
  model.load_state_dict(torch.load(MODEL_PATH))
#   tokenizer = DistilBertTokenizer.from_pretrained(VOCAB_PATH)

  return model #, tokenizer

In [9]:
def saveCSVValidationResult(predictions, path):
    path = path+"/Topic(Seller)2SubTopic/back_only_01/result.csv"
    with open(path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Text', 'Predicted', 'Target'])
        for example in predictions:
            writer.writerow([example['text'], example['predicted'], example['target']])